In [11]:
import torch
import torchvision
import onnx
import torch.nn as nn
import torch.nn.functional as F

In [13]:
torch.manual_seed(1)

In [29]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension(批大小维度)
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [30]:
params = list(net.parameters())
print(len(params))
print(type(params[0]))
for i in range(10):
    print("第{}个参数形状为{}".format(i,params[i].size()))

10
<class 'torch.nn.parameter.Parameter'>
第0个参数形状为torch.Size([6, 1, 5, 5])
第1个参数形状为torch.Size([6])
第2个参数形状为torch.Size([16, 6, 5, 5])
第3个参数形状为torch.Size([16])
第4个参数形状为torch.Size([120, 400])
第5个参数形状为torch.Size([120])
第6个参数形状为torch.Size([84, 120])
第7个参数形状为torch.Size([84])
第8个参数形状为torch.Size([10, 84])
第9个参数形状为torch.Size([10])


In [35]:
input = torch.randn(1, 1, 32, 32)
print(input)
print('conv1输出的形状：',net.conv1(input).size()) # conv1输出的形状

out = net(input)
print('输出层形状',out.size())
print('输出层结果：',out)

tensor([[[[-0.5325,  0.3985,  1.4077,  ...,  0.9968, -0.5491,  1.6251],
          [ 1.3229,  0.7215,  1.4408,  ...,  1.9131, -0.3747, -1.1335],
          [-0.1019, -1.8693,  0.0185,  ..., -0.9244, -0.7346,  0.7157],
          ...,
          [-0.1353,  1.8613,  0.6069,  ..., -1.3562, -1.3796,  0.8008],
          [ 0.3595,  1.2930, -1.3521,  ...,  2.0284,  1.3408, -0.8612],
          [ 0.2900,  1.9590,  1.9590,  ...,  0.5073,  0.6317,  0.1085]]]])
conv1输出的形状： torch.Size([1, 6, 28, 28])
输出层形状 torch.Size([1, 10])
输出层结果： tensor([[-0.7075,  0.5646, -0.1979,  0.7789,  0.7454, -0.1606,  0.6437,  0.1402,
         -0.2897, -0.2493]], grad_fn=<AddmmBackward>)


In [36]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss() 


loss = criterion(output, target)
print('loss:',loss)
print('''
反向跟踪loss,使用它的.grad_fn属性,你会看到向下面这样的一个计算图:
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d -> view -> linear -> relu -> linear -> relu -> linear -> MSELoss -> loss''')
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

loss: tensor(1.4125, grad_fn=<MseLossBackward>)

反向跟踪loss,使用它的.grad_fn属性,你会看到向下面这样的一个计算图:
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d -> view -> linear -> relu -> linear -> relu -> linear -> MSELoss -> loss


In [37]:
# 优化器
optimizer = torch.optim.SGD(net.parameters(),lr=0.1)

In [44]:
for epoch in range(10):
    optimizer.zero_grad()   # zero the gradient buffers
    output = net(input)
    loss = criterion(output, target)
    print(loss)
    loss.backward()
    optimizer.step()    # Does the update
    
torch.save(net, 'net.pth')  # 保存整个网络
torch.save(net.state_dict(), 'net_params.pth')   # 只保存网络中的参数 (速度快, 占内存少)

tensor(0.0437, grad_fn=<MseLossBackward>)
tensor(0.1659, grad_fn=<MseLossBackward>)
tensor(0.0755, grad_fn=<MseLossBackward>)
tensor(0.2684, grad_fn=<MseLossBackward>)
tensor(0.0087, grad_fn=<MseLossBackward>)
tensor(0.0294, grad_fn=<MseLossBackward>)
tensor(0.0556, grad_fn=<MseLossBackward>)
tensor(0.1909, grad_fn=<MseLossBackward>)
tensor(0.0367, grad_fn=<MseLossBackward>)
tensor(0.1223, grad_fn=<MseLossBackward>)


/home/xiang/anaconda3/envs/py37/lib/python3.7/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [45]:
input2 = torch.ones(1,1,32,32)
print(net(input2))

tensor([[ 0.7437, -0.9348, -0.2768, -0.7745,  0.6113,  2.3297,  1.3693,  0.1077,
          1.6619,  0.9809]], grad_fn=<AddmmBackward>)
